# Using the OpenAI API
* **Created by:** Eric Martinez
* **For:** DDAMC

## OpenAI API

The OpenAI API provides access to powerful LLMs like GPT-3.5 and GPT-4, enabling developers to leverage these models in their applications. To access the API, sign up for an API key on the OpenAI website and follow the documentation to make API calls.

For enterprise: Azure OpenAI offers a robust and scalable platform for deploying LLMs in enterprise applications. It provides features like security, compliance, and support, making it an ideal choice for businesses looking to leverage LLMs.

Options:
* [[Paid] Sign-up for OpenAI API Access](https://platform.openai.com/signup)

## Managing Application Secrets

Secrets are sensitive information, such as API keys, passwords, or cryptographic keys, that must be protected to ensure the security and integrity of a system.

In software development, secrets are often used to authenticate users, grant access to resources, or encrypt/decrypt data. Mismanaging or exposing secrets can lead to severe security breaches and data leaks.

#### Common examples of secrets
* API keys
* Database credentials
* SSH keys
* OAuth access tokens
* Encryption/decryption keys

#### Common mistakes when handling secrets
* Storing secrets in plain text
* Hardcoding secrets in source code
* Sharing secrets through unsecured channels (e.g., email or messaging apps)
* Using the same secret for multiple purposes
* Not rotating or updating secrets regularly

How attackers might obtain secrets
* Exploiting vulnerabilities in software or infrastructure
* Intercepting unencrypted communications
* Gaining unauthorized access to repositories or storage systems
* Social engineering or phishing attacks
* Brute-forcing weak secrets

The impact of compromised secrets
* Unauthorized access to sensitive data
* Data breaches, resulting in financial loss and reputational damage
* Loss of trust from customers and stakeholders
* Legal repercussions and regulatory fines
* Potential takeover or manipulation of systems

Steps to protect secrets
* Store secrets securely using secret management tools or dedicated secret storage services
* Encrypt secrets at rest and in transit
* Use strong, unique secrets and rotate them regularly
* Limit access to secrets on a need-to-know basis
* Implement proper auditing and monitoring of secret usage

Cloud services and secret management
* Many cloud providers offer secret management services (e.g., AWS Secrets Manager, Azure Key Vault, Google Cloud Secret Manager) that securely store, manage, and rotate secrets.
* These services often provide access control, encryption, and auditing capabilities.
* Integrating cloud secret management services with your application can help secure sensitive information and reduce the risk of exposure.

Best practices for secrets
* Use different secrets for development, testing, and production environments to minimize the risk of accidental exposure.
* Regularly audit and review secret access to ensure only authorized users have access.
* Establish a clear process for managing secrets, including secret creation, storage, rotation, and deletion.
* Educate team members on the importance of secret security and the best practices for handling sensitive information.

#### Using  `.dotenv` library to protect secrets in Python

 `.dotenv` is a Python library that allows developers to load environment variables from a `.env` file. It helps keep secrets out of source code and makes it easier to manage and update them.

Install the `python-dotenv` library

In [7]:
!pip -q install --upgrade python-dotenv

Create a `.env` file in this folder using any editor.

Add secrets as key-value pairs in the `.env` file

Use the following format:

In [ ]:
OPENAI_API_KEY=<your OpenAI API key>

Then, use the following code to load those secrets into this notebook:

In [2]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

#### Install Dependencies

In [8]:
!pip -q install --upgrade openai

#### Let's make a function to wrap OpenAI functionality and write some basic tests

Start by simply seeing if we can make an API call

In [3]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

import openai

# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-4", system_message=None, temperature=0):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]
    
    # Add the user's message to the messages list
    messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

prompt = "When I say hello, you say world"
print(get_ai_reply("hello", system_message=prompt))

world


Check to see if we can get the data as a CSV formatted string

In [5]:

prompt="""
You are an expert in medical transcription.

## Task

You will be provided with unstructured data, and your task is to parse it into CSV format.

## Format 

The headers for your CSV output are as follows:
patient_name, age, symptoms, duration, additional_information

Always include the header again in your response.

## Example

User: "Patient John, aged 42 years, reported a stiff neck and a constant, throbbing headache for the past three days. Symptoms are severe in the morning. Patient Mary, 56 years old, complained about abdominal pain and vomiting for the last two days. Pain is severe and the vomiting is recurring after meals. Young patient Timothy, 7 years old, has been experiencing mild fever and a runny nose for five days, symptoms are worse in the evening. Lastly, patient Lisa, 36 years old, reported chronic low back pain for two weeks, the pain is moderate and often increases with physical activity."

Assistant:
patient_name, age, symptoms, duration, additional_information
John,42,"Stiff neck and constant, throbbing headache",Three days,Severe in the morning
Mary,56,Abdominal pain and vomiting,Two days,Severe and recurring after meals
Timothy,7,Mild fever and a runny nose,Five days,Worse in the evening
Lisa,36,Chronic low back pain,Two weeks,Moderate and increases with physical activity

## Begin
"""

input = "Resident Noah, a 65-year-old, has had hypertension and intermittent chest discomfort for the last week. Discomfort is particularly noticeable after physical activity. Emma, a 25-year-old, has reported a rash and joint pain that began three days ago and seems to increase in severity at night. Another patient, Lucas, aged 33, has been experiencing frequent urination and weight loss over the past month, with mild discomfort reported. Finally, Olivia, a 48-year-old, has a recurring dry cough and feeling of breathlessness for the last two weeks, primarily occurring during the daytime."
csv_text = get_ai_reply(input, system_message=prompt) # get csv from openai
print(csv_text)


patient_name, age, symptoms, duration, additional_information
Noah,65,Hypertension and intermittent chest discomfort,One week,Noticeable after physical activity
Emma,25,Rash and joint pain,Three days,Increases in severity at night
Lucas,33,Frequent urination and weight loss,One month,Mild discomfort
Olivia,48,Recurring dry cough and breathlessness,Two weeks,Primarily during the daytime


Create a function that will do all of this for us and return a Pandas dataframe

In [31]:
import pandas as pd
import io   

def df_from_text(text):
    prompt="""
    You are an expert in medical transcription.

    ## Task

    You will be provided with unstructured data, and your task is to parse it into CSV format.

    ## Format 

    The headers for your CSV output are as follows:
    patient_name, age, symptoms, duration, additional_information

    Always include the header again in your response.

    ## Example

    User: "Patient John, aged 42 years, reported a stiff neck and a constant, throbbing headache for the past three days. Symptoms are severe in the morning. Patient Mary, 56 years old, complained about abdominal pain and vomiting for the last two days. Pain is severe and the vomiting is recurring after meals. Young patient Timothy, 7 years old, has been experiencing mild fever and a runny nose for five days, symptoms are worse in the evening. Lastly, patient Lisa, 36 years old, reported chronic low back pain for two weeks, the pain is moderate and often increases with physical activity."

    Assistant:
    patient_name, age, symptoms, duration, additional_information
    John,42,"Stiff neck and constant, throbbing headache",Three days,Severe in the morning
    Mary,56,Abdominal pain and vomiting,Two days,Severe and recurring after meals
    Timothy,7,Mild fever and a runny nose,Five days,Worse in the evening
    Lisa,36,Chronic low back pain,Two weeks,Moderate and increases with physical activity

    ## Begin
    """

    csv_text = get_ai_reply(text, system_message=prompt) # get csv from openai
    df = pd.read_csv(io.StringIO(csv_text)) # convert to pandas dataframe
    return df


In [35]:
text = "Resident Noah, a 65-year-old, has had hypertension and intermittent chest discomfort for the last week. Discomfort is particularly noticeable after physical activity. Emma, a 25-year-old, has reported a rash and joint pain that began three days ago and seems to increase in severity at night. Another patient, Lucas, aged 33, has been experiencing frequent urination and weight loss over the past month, with mild discomfort reported. Finally, Olivia, a 48-year-old, has a recurring dry cough and feeling of breathlessness for the last two weeks, primarily occurring during the daytime."
df = df_from_text(text)
display(df)

,patient_name,age,symptoms,duration,additional_information
0,Noah,65,Hypertension and intermittent chest discomfort,One week,Noticeable after physical activity
1,Emma,25,Rash and joint pain,Three days,Increases in severity at night
2,Lucas,33,Frequent urination and weight loss,One month,Mild discomfort
3,Olivia,48,Recurring dry cough and breathlessness,Two weeks,Primarily during the daytime


Create a user interface for this using Gradio

In [33]:
!pip -q install --upgrade gradio

In [34]:
import gradio as gr

def parse_text(text):
    df = df_from_text(text)
    csv_location = "output.csv"
    df.to_csv(csv_location)
    return df, csv_location

with gr.Blocks() as demo:
    with gr.Tab("Unstructured Notes to CSV"):
        with gr.Row():
            with gr.Column():
                notes = gr.Textbox(label="Notes", lines=5)
                btn = gr.Button(value ="Submit")
                table = gr.Dataframe(label="Results")
                csv_file = gr.File(label="CSV Output", interactive=False)
            btn.click(parse_text, inputs = [notes], outputs = [table, csv_file])
    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://9dd0d51dadd9efee1d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
